### 2RP Net - Data Engineer Test

## 6.	Gere scripts que atendam as solicitações abaixo:

In [1]:
# Import package
try:
    import pandas as pd  # Import pandas
    import numpy as np
    import pandera as pa
    import os
    import sys
    import glob
    import time
    import urllib.request
    from urllib.request import urlretrieve
except Exception as e:
    print("Error : {'Falha Imports'} ".format(e))

In [2]:
# Marca o horário do início do programa
s_time_control = time.time()

#### Concaternar arquivos e redução de tamanho em conversão para parquet

In [3]:
os.chdir("/Jupyter/2RP")  

In [4]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [5]:
#combinar todos os arquivos da lista
df = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [6]:
e_time_dask = time.time()
print("Tempo de criação do arquivo: ", round(e_time_dask-s_time_control)/60, "minutes")

Tempo de criação do arquivo:  6.233333333333333 minutes


In [7]:
df.head(3)

,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,ICB_NAME,ICB_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,ADDRESS_1,...,BNF_CODE,BNF_DESCRIPTION,BNF_CHAPTER_PLUS_CODE,QUANTITY,ITEMS,TOTAL_QUANTITY,ADQUSAGE,NIC,ACTUAL_COST,UNIDENTIFIED
0,202206,NORTH WEST,Y62,NHS CHESHIRE AND MERSEYSIDE INTEGRATED C,QYG,WIRRAL COMMUNITY HEALTH AND CARE NHS FOU,RY700,WIRRAL COMMUNITY NMP,Y03836,ST CATHERINE'S HC,...,20020200701,Viscopaste PB7 bandage 7.5cm x 6m,20: Dressings,10.0,1,10.0,0.0,38.90,36.40326,N
1,202206,NORTH WEST,Y62,NHS CHESHIRE AND MERSEYSIDE INTEGRATED C,QYG,WIRRAL COMMUNITY HEALTH AND CARE NHS FOU,RY700,WIRRAL WIC (APH)_WIC APH,N85645,ARROWE PARK HOSPITAL,...,20030100079,Mepore dressing 11cm x 15cm,20: Dressings,5.0,1,5.0,0.0,1.85,1.74307,N
2,202206,NORTH EAST AND YORKSHIRE,Y63,NHS SOUTH YORKSHIRE INTEGRATED CARE BOAR,QF7,NHS NOTTINGHAM AND NOTTINGHAMSHIRE ICB -,02Q00,BASSETLAW HEALTH PARTNERSHIP,Y03762,C/O RETFORD HOSPITAL,...,20030100167,Dressit sterile dressing pack with gloves,20: Dressings,10.0,6,60.0,0.0,41.40,38.75440,N


In [8]:
df.keys()

Index(['YEAR_MONTH', 'REGIONAL_OFFICE_NAME', 'REGIONAL_OFFICE_CODE',
       'ICB_NAME', 'ICB_CODE', 'PCO_NAME', 'PCO_CODE', 'PRACTICE_NAME',
       'PRACTICE_CODE', 'ADDRESS_1', 'ADDRESS_2', 'ADDRESS_3', 'ADDRESS_4',
       'POSTCODE', 'BNF_CHEMICAL_SUBSTANCE', 'CHEMICAL_SUBSTANCE_BNF_DESCR',
       'BNF_CODE', 'BNF_DESCRIPTION', 'BNF_CHAPTER_PLUS_CODE', 'QUANTITY',
       'ITEMS', 'TOTAL_QUANTITY', 'ADQUSAGE', 'NIC', 'ACTUAL_COST',
       'UNIDENTIFIED'],
      dtype='object')

In [9]:
df['REGIONAL_OFFICE_CODE'].unique()

array(['Y62', 'Y63', 'Y58', 'Y59', 'Y61', 'Y56', 'Y60', '-'], dtype=object)

In [10]:
df['REGIONAL_OFFICE_NAME'].unique()

array(['NORTH WEST', 'NORTH EAST AND YORKSHIRE', 'SOUTH WEST',
       'SOUTH EAST', 'EAST OF ENGLAND', 'LONDON', 'MIDLANDS',
       'UNIDENTIFIED'], dtype=object)

In [11]:
df[['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE']].groupby(['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE']).head(10)

,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE
0,NORTH WEST,Y62
1,NORTH WEST,Y62
2,NORTH EAST AND YORKSHIRE,Y63
3,NORTH EAST AND YORKSHIRE,Y63
4,NORTH EAST AND YORKSHIRE,Y63
...,...,...
6550461,UNIDENTIFIED,-
6550462,UNIDENTIFIED,-
6550463,UNIDENTIFIED,-
6550464,UNIDENTIFIED,-


#### a.	Crie um dataframe contendo os 10 principais produtos químicos prescritos por região.

In [12]:
df1 = df.groupby('REGIONAL_OFFICE_NAME').get_group('NORTH WEST')

In [13]:
df_Y62 = df1[['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR', 'TOTAL_QUANTITY']]

In [14]:
df_Y62.groupby(['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR']).TOTAL_QUANTITY.sum().reset_index().sort_values('TOTAL_QUANTITY',ascending=False).head(10)

,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,CHEMICAL_SUBSTANCE_BNF_DESCR,TOTAL_QUANTITY
384,NORTH WEST,Y62,Enteral nutrition,1.055717e+09
374,NORTH WEST,Y62,Emollients,1.439674e+08
839,NORTH WEST,Y62,Other emollient preparations,1.214324e+08
843,NORTH WEST,Y62,Other food for special diet preparations,1.039796e+08
31,NORTH WEST,Y62,Alginic acid compound preparations,9.426590e+07
889,NORTH WEST,Y62,Paracetamol,8.966254e+07
249,NORTH WEST,Y62,Co-codamol (Codeine phosphate/paracetamol),8.351371e+07
720,NORTH WEST,Y62,Metformin hydrochloride,6.821325e+07
86,NORTH WEST,Y62,Atorvastatin,6.161963e+07
615,NORTH WEST,Y62,Lactulose,5.926078e+07


In [15]:
df1 = df.groupby('REGIONAL_OFFICE_NAME').get_group('NORTH EAST AND YORKSHIRE')

In [16]:
df_Y63 = df1[['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR', 'TOTAL_QUANTITY']]

In [17]:
df_Y63.groupby(['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR']).TOTAL_QUANTITY.sum().reset_index().sort_values('TOTAL_QUANTITY',ascending=False).head(10)

,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,CHEMICAL_SUBSTANCE_BNF_DESCR,TOTAL_QUANTITY
397,NORTH EAST AND YORKSHIRE,Y63,Enteral nutrition,940501371.0
386,NORTH EAST AND YORKSHIRE,Y63,Emollients,189867018.0
864,NORTH EAST AND YORKSHIRE,Y63,Other food for special diet preparations,140280360.0
907,NORTH EAST AND YORKSHIRE,Y63,Paracetamol,139349715.0
35,NORTH EAST AND YORKSHIRE,Y63,Alginic acid compound preparations,131919015.0
860,NORTH EAST AND YORKSHIRE,Y63,Other emollient preparations,105236721.0
733,NORTH EAST AND YORKSHIRE,Y63,Metformin hydrochloride,81777144.0
90,NORTH EAST AND YORKSHIRE,Y63,Atorvastatin,79423329.0
737,NORTH EAST AND YORKSHIRE,Y63,Methadone hydrochloride,71518446.0
257,NORTH EAST AND YORKSHIRE,Y63,Co-codamol (Codeine phosphate/paracetamol),66996174.0


In [18]:
df1 = df.groupby('REGIONAL_OFFICE_NAME').get_group('SOUTH WEST')

In [19]:
df_Y58 = df1[['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR', 'TOTAL_QUANTITY']]

In [20]:
df_Y58.groupby(['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR']).TOTAL_QUANTITY.sum().reset_index().sort_values('TOTAL_QUANTITY',ascending=False).head(10)

,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,CHEMICAL_SUBSTANCE_BNF_DESCR,TOTAL_QUANTITY
366,SOUTH WEST,Y58,Enteral nutrition,351641637.0
355,SOUTH WEST,Y58,Emollients,120679149.0
854,SOUTH WEST,Y58,Paracetamol,77273145.0
815,SOUTH WEST,Y58,Other food for special diet preparations,51360490.5
789,SOUTH WEST,Y58,Omeprazole,46906782.0
811,SOUTH WEST,Y58,Other emollient preparations,46831140.0
31,SOUTH WEST,Y58,Alginic acid compound preparations,46516143.0
81,SOUTH WEST,Y58,Atorvastatin,46198941.0
694,SOUTH WEST,Y58,Metformin hydrochloride,45163596.0
238,SOUTH WEST,Y58,Co-codamol (Codeine phosphate/paracetamol),39139740.0


In [21]:
df1 = df.groupby('REGIONAL_OFFICE_NAME').get_group('SOUTH EAST')

In [22]:
df_Y59 = df1[['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR', 'TOTAL_QUANTITY']]

In [23]:
df_Y59.groupby(['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR']).TOTAL_QUANTITY.sum().reset_index().sort_values('TOTAL_QUANTITY',ascending=False).head(10)

,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,CHEMICAL_SUBSTANCE_BNF_DESCR,TOTAL_QUANTITY
384,SOUTH EAST,Y59,Enteral nutrition,790334163.0
373,SOUTH EAST,Y59,Emollients,166381674.0
854,SOUTH EAST,Y59,Other food for special diet preparations,116736750.0
733,SOUTH EAST,Y59,Methadone hydrochloride,102929214.0
899,SOUTH EAST,Y59,Paracetamol,79587024.0
850,SOUTH EAST,Y59,Other emollient preparations,75245883.0
729,SOUTH EAST,Y59,Metformin hydrochloride,74873643.0
84,SOUTH EAST,Y59,Atorvastatin,64631658.0
827,SOUTH EAST,Y59,Omeprazole,57593217.0
248,SOUTH EAST,Y59,Co-codamol (Codeine phosphate/paracetamol),57434982.0


In [24]:
df1 = df.groupby('REGIONAL_OFFICE_NAME').get_group('EAST OF ENGLAND')

In [25]:
df_Y61 = df1[['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR', 'TOTAL_QUANTITY']]

In [26]:
df_Y61.groupby(['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR']).TOTAL_QUANTITY.sum().reset_index().sort_values('TOTAL_QUANTITY',ascending=False).head(10)

,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,CHEMICAL_SUBSTANCE_BNF_DESCR,TOTAL_QUANTITY
384,EAST OF ENGLAND,Y61,Enteral nutrition,581225739.0
373,EAST OF ENGLAND,Y61,Emollients,123782622.0
848,EAST OF ENGLAND,Y61,Other food for special diet preparations,76037229.0
721,EAST OF ENGLAND,Y61,Metformin hydrochloride,58428774.0
889,EAST OF ENGLAND,Y61,Paracetamol,57713838.0
845,EAST OF ENGLAND,Y61,Other emollient preparations,52460835.0
84,EAST OF ENGLAND,Y61,Atorvastatin,52323834.0
247,EAST OF ENGLAND,Y61,Co-codamol (Codeine phosphate/paracetamol),46100589.0
33,EAST OF ENGLAND,Y61,Alginic acid compound preparations,40360749.0
613,EAST OF ENGLAND,Y61,Lactulose,38576361.0


In [27]:
df1 = df.groupby('REGIONAL_OFFICE_NAME').get_group('LONDON')

In [28]:
df_Y56 = df1[['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR', 'TOTAL_QUANTITY']]

In [29]:
df_Y56.groupby(['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR']).TOTAL_QUANTITY.sum().reset_index().sort_values('TOTAL_QUANTITY',ascending=False).head(10)

,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,CHEMICAL_SUBSTANCE_BNF_DESCR,TOTAL_QUANTITY
387,LONDON,Y56,Enteral nutrition,872748183.0
375,LONDON,Y56,Emollients,217517643.0
865,LONDON,Y56,Other emollient preparations,98178603.0
738,LONDON,Y56,Metformin hydrochloride,95322528.0
869,LONDON,Y56,Other food for special diet preparations,79515051.0
84,LONDON,Y56,Atorvastatin,60796728.0
742,LONDON,Y56,Methadone hydrochloride,54174177.0
32,LONDON,Y56,Alginic acid compound preparations,50628864.0
913,LONDON,Y56,Paracetamol,48319713.0
56,LONDON,Y56,Amlodipine,43952670.0


In [30]:
df1 = df.groupby('REGIONAL_OFFICE_NAME').get_group('MIDLANDS')

In [31]:
df_indef = df1[['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR', 'TOTAL_QUANTITY']]

In [32]:
df_indef.groupby(['REGIONAL_OFFICE_NAME','REGIONAL_OFFICE_CODE','CHEMICAL_SUBSTANCE_BNF_DESCR']).TOTAL_QUANTITY.sum().reset_index().sort_values('TOTAL_QUANTITY',ascending=False).head(10)

,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,CHEMICAL_SUBSTANCE_BNF_DESCR,TOTAL_QUANTITY
391,MIDLANDS,Y60,Enteral nutrition,1.233260e+09
380,MIDLANDS,Y60,Emollients,2.499766e+08
869,MIDLANDS,Y60,Other food for special diet preparations,1.736317e+08
865,MIDLANDS,Y60,Other emollient preparations,1.359163e+08
915,MIDLANDS,Y60,Paracetamol,1.337383e+08
741,MIDLANDS,Y60,Metformin hydrochloride,1.070606e+08
34,MIDLANDS,Y60,Alginic acid compound preparations,1.044320e+08
257,MIDLANDS,Y60,Co-codamol (Codeine phosphate/paracetamol),9.989350e+07
87,MIDLANDS,Y60,Atorvastatin,8.912797e+07
633,MIDLANDS,Y60,Lactulose,7.846576e+07


#### b.	Quais produtos químicos prescritos tiveram a maior somatória de custos por mês? Liste os 10 primeiros.

In [33]:
df.groupby("CHEMICAL_SUBSTANCE_BNF_DESCR").ACTUAL_COST.sum().reset_index().sort_values('ACTUAL_COST',ascending=False).head(10)

,CHEMICAL_SUBSTANCE_BNF_DESCR,ACTUAL_COST
77,Apixaban,9.902231e+07
439,Enteral nutrition,7.308612e+07
114,Beclometasone dipropionate,7.133825e+07
1176,Rivaroxaban,5.634314e+07
208,Catheters,3.886361e+07
352,Detection Sensor Interstitial Fluid/Gluc,3.273340e+07
164,Budesonide,2.919755e+07
1421,Wound Management & Other Dressings,2.876698e+07
574,Glucose blood testing reagents,2.769182e+07
419,Edoxaban,2.736962e+07


#### c.	Quais são as precrições mais comuns? 

In [34]:
df[['BNF_DESCRIPTION','TOTAL_QUANTITY']].groupby('BNF_DESCRIPTION').sum().sort_values(by='TOTAL_QUANTITY', ascending=False).head(10)

,TOTAL_QUANTITY
BNF_DESCRIPTION,
Ensure Plus milkshake style liquid (9 flavours),562623000.0
Paracetamol 500mg tablets,458637687.0
Fortisip Bottle (8 flavours),454686000.0
Fortisip Compact liquid (8 flavours),418508250.0
Ensure Compact liquid (4 flavours),374188584.0
Lactulose 3.1-3.7g/5ml oral solution,345280785.0
Metformin 500mg tablets,334258011.0
Fortisip Compact Protein liquid (9 flavours),321539250.0
Omeprazole 20mg gastro-resistant capsules,299100483.0


#### d.	Qual produto químico é mais prescrito por cada prescriber?

In [35]:
df_pres = df[['PRACTICE_NAME','CHEMICAL_SUBSTANCE_BNF_DESCR','TOTAL_QUANTITY']]

In [36]:
df_asc = df_pres.groupby(['PRACTICE_NAME','CHEMICAL_SUBSTANCE_BNF_DESCR']).TOTAL_QUANTITY.sum().reset_index().sort_values('TOTAL_QUANTITY',ascending=False)

In [37]:
df_prescriber = df_asc.drop_duplicates('PRACTICE_NAME')

In [38]:
df_prescriber.sort_values('TOTAL_QUANTITY',ascending=False)

,PRACTICE_NAME,CHEMICAL_SUBSTANCE_BNF_DESCR,TOTAL_QUANTITY
1548621,MEDICUS HEALTH PARTNERS,Enteral nutrition,12760896.0
443167,CGL BIRMINGHAM SOUTH,Methadone hydrochloride,11910675.0
1600962,MODALITY PARTNERSHIP (AWC),Enteral nutrition,11847435.0
1573996,MIDLANDS MEDICAL PARTNERSHIP,Enteral nutrition,10096872.0
1812476,PARK SURGERY,Enteral nutrition,9874746.0
...,...,...,...
2023019,ROTHERHAM DISTRICT NURSING,Eye Products,3.0
2356363,TAMWORTH LOCALITY NETWORK,Salbutamol,3.0
995523,GPSI CYSTOSCOPY - IDLE,Testosterone esters,3.0
1727755,NT&H COMM DIAB N DDES,Glucagon,3.0


#### e.	Quantos prescribers foram adicionados no ultimo mês? 

Realizei a analise pela a otica dos precriber que no caso são os centros de saúde e não houve aumento de unidades ou seja o número é fixo de prescribers. 

In [39]:
df_202207 = pd.read_csv('epd_202207.csv')

In [40]:
df_prescribers = df_202207[['PRACTICE_NAME']]

In [41]:
print(df_prescribers.groupby('PRACTICE_NAME').PRACTICE_NAME.sum().count().sum(),'Centros de prescrições - Centros de tratamentos - Hospitais')

8387 Centros de prescrições - Centros de tratamentos - Hospitais


In [42]:
df_prescriptions = df_202207[['BNF_DESCRIPTION']]

In [43]:
print(df_prescriptions.groupby('BNF_DESCRIPTION').BNF_DESCRIPTION.count().sum(),'prescrições realizadas')

17603900 prescrições realizadas


#### f.	Quais prescribers atuam em mais de uma região? Ordene por quantidade de regiões antendidas.

#### Códigos realizados com a finalidade de conhecer melhor a base de dados

In [44]:
df_region = df[['PRACTICE_NAME','PRACTICE_CODE','REGIONAL_OFFICE_NAME']]

In [45]:
df_region

,PRACTICE_NAME,PRACTICE_CODE,REGIONAL_OFFICE_NAME
0,WIRRAL COMMUNITY NMP,Y03836,NORTH WEST
1,WIRRAL WIC (APH)_WIC APH,N85645,NORTH WEST
2,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
3,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
4,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
...,...,...,...
17603895,THE RISE GROUP PRACTICE,F83039,LONDON
17603896,THE RISE GROUP PRACTICE,F83039,LONDON
17603897,THE RISE GROUP PRACTICE,F83039,LONDON
17603898,THE RISE GROUP PRACTICE,F83039,LONDON


In [46]:
df_region.value_counts()

PRACTICE_NAME                            PRACTICE_CODE  REGIONAL_OFFICE_NAME    
MODALITY PARTNERSHIP (AWC)               B83033         NORTH EAST AND YORKSHIRE    35049
MEDICUS HEALTH PARTNERS                  F85002         LONDON                      30201
MIDLANDS MEDICAL PARTNERSHIP             M85063         MIDLANDS                    29946
SHORE MEDICAL                            J81012         SOUTH WEST                  25506
PORTSDOWN GROUP PRACTICE                 J82155         SOUTH EAST                  24696
                                                                                    ...  
TRAFFORD DRUG SERVICE                    Y00480         NORTH WEST                      3
HAWTHORNS SPINAL UNIT                    Y03574         NORTH EAST AND YORKSHIRE        3
ERP EAST                                 Y01701         NORTH EAST AND YORKSHIRE        3
CHILD & FAMILY COMMUNITY PAEDIATRICIANS  Y05299         MIDLANDS                        3
HMP EAST SUTTON PAR

In [47]:
df_region.drop_duplicates()

,PRACTICE_NAME,PRACTICE_CODE,REGIONAL_OFFICE_NAME
0,WIRRAL COMMUNITY NMP,Y03836,NORTH WEST
1,WIRRAL WIC (APH)_WIC APH,N85645,NORTH WEST
2,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
34,GP APH OOH,N85638,NORTH WEST
41,SEVERNSIDE MEDICAL PRACTICE,L84052,SOUTH WEST
...,...,...,...
17582508,THE PRACTICE AT 188,E83027,LONDON
17589495,QUEEN STREET SURGERY,B87600,NORTH EAST AND YORKSHIRE
17594786,GP LED HEALTH CENTRE,Y02854,NORTH WEST
17595508,THE REGENTS PARK PRACTICE,F83025,LONDON


In [48]:
df_region.groupby(['PRACTICE_NAME','PRACTICE_CODE','REGIONAL_OFFICE_NAME']).sum().head(8690)

,,
PRACTICE_NAME,PRACTICE_CODE,REGIONAL_OFFICE_NAME
(FRACTURE CLINIC) NORTH OOH,Y00082,MIDLANDS
(IRLAM) SALFORD CARE CTRS MEDICAL PRACTI,P87657,NORTH WEST
(OUT PATIENT DEPARTMENT) NORTH OOH,Y00234,MIDLANDS
0-19 EAST CHESHIRE HEALTH VISITORS,Y05381,NORTH WEST
0-19 PUBLIC HEALTH SERVICE HARTLEPOOL,Y04082,NORTH EAST AND YORKSHIRE
...,...,...
WIGSTON CENTRAL SURGERY,C82071,MIDLANDS
WIGTON GROUP MEDICAL PRACTICE,A82045,NORTH EAST AND YORKSHIRE
WILBERFORCE SURGERY,B81032,NORTH EAST AND YORKSHIRE


In [49]:
df_region.groupby(['PRACTICE_NAME','PRACTICE_CODE','REGIONAL_OFFICE_NAME']).count().head(8690)

,,
PRACTICE_NAME,PRACTICE_CODE,REGIONAL_OFFICE_NAME
(FRACTURE CLINIC) NORTH OOH,Y00082,MIDLANDS
(IRLAM) SALFORD CARE CTRS MEDICAL PRACTI,P87657,NORTH WEST
(OUT PATIENT DEPARTMENT) NORTH OOH,Y00234,MIDLANDS
0-19 EAST CHESHIRE HEALTH VISITORS,Y05381,NORTH WEST
0-19 PUBLIC HEALTH SERVICE HARTLEPOOL,Y04082,NORTH EAST AND YORKSHIRE
...,...,...
WIGSTON CENTRAL SURGERY,C82071,MIDLANDS
WIGTON GROUP MEDICAL PRACTICE,A82045,NORTH EAST AND YORKSHIRE
WILBERFORCE SURGERY,B81032,NORTH EAST AND YORKSHIRE


In [50]:
df_pract = df_region.groupby(['PRACTICE_CODE']).count()

In [51]:
df_pract

,PRACTICE_NAME,REGIONAL_OFFICE_NAME
PRACTICE_CODE,,
-,19227,19227
00L998,3,3
00P998,99,99
00Q998,495,495
00R998,576,576
...,...,...
Y07329,2040,2040
Y07332,6,6
Y07343,6,6


In [52]:
df_pract.groupby(['PRACTICE_NAME']).sum().count().sum()

3646

In [53]:
df_duplic = df_pract.sort_values('PRACTICE_NAME',ascending=True).duplicated()

In [54]:
df_duplic

PRACTICE_CODE
Y07362    False
Y06616     True
Y03941     True
Y06635     True
Y06638     True
          ...  
J82155    False
J81012    False
M85063    False
F85002    False
B83033    False
Length: 8927, dtype: bool

In [55]:
df_duplic.sum()

5281

In [56]:
df_duplic.count()

8927

In [57]:
# Observa-se que na verdade os Prescribers mesmo possuíndo o mesmo nome
df_region.drop_duplicates()

,PRACTICE_NAME,PRACTICE_CODE,REGIONAL_OFFICE_NAME
0,WIRRAL COMMUNITY NMP,Y03836,NORTH WEST
1,WIRRAL WIC (APH)_WIC APH,N85645,NORTH WEST
2,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
34,GP APH OOH,N85638,NORTH WEST
41,SEVERNSIDE MEDICAL PRACTICE,L84052,SOUTH WEST
...,...,...,...
17582508,THE PRACTICE AT 188,E83027,LONDON
17589495,QUEEN STREET SURGERY,B87600,NORTH EAST AND YORKSHIRE
17594786,GP LED HEALTH CENTRE,Y02854,NORTH WEST
17595508,THE REGENTS PARK PRACTICE,F83025,LONDON


In [58]:
# Imprime a(s) linha(s) duplicadas
df_region[df_region.duplicated()]

,PRACTICE_NAME,PRACTICE_CODE,REGIONAL_OFFICE_NAME
3,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
4,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
5,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
6,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
7,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
...,...,...,...
17603895,THE RISE GROUP PRACTICE,F83039,LONDON
17603896,THE RISE GROUP PRACTICE,F83039,LONDON
17603897,THE RISE GROUP PRACTICE,F83039,LONDON
17603898,THE RISE GROUP PRACTICE,F83039,LONDON


In [59]:
# Neste modo de visualização pode ser observado  que os Prescribers (PRACTICE_NAME) com o seu identificador (PRACTICE_CODE) que todos os Prescribers em diferentes regiões mesmo possuindo nomes iguais e identificadores diferentes.
# Imprime a(s) linha(s) duplicadas
df_region[df_region.duplicated()].drop_duplicates()

,PRACTICE_NAME,PRACTICE_CODE,REGIONAL_OFFICE_NAME
3,BASSETLAW HEALTH PARTNERSHIP,Y03762,NORTH EAST AND YORKSHIRE
29,WIRRAL WIC (APH)_WIC APH,N85645,NORTH WEST
31,WIRRAL COMMUNITY NMP,Y03836,NORTH WEST
42,SEVERNSIDE MEDICAL PRACTICE,L84052,SOUTH WEST
45,SEVERNBANK SURGERY,L84085,SOUTH WEST
...,...,...,...
16531666,COMMUNITY STROKE REHABILITATION TEAM,Y06520,NORTH EAST AND YORKSHIRE
16568317,WEYBRIDGE HOSP & PRIMARY CARE CTR WIC,H81669,SOUTH EAST
16862946,CENTRAL MAPLE PCN,Y07362,SOUTH EAST
17307116,ADULT COMMUNITY NURSING,Y04317,SOUTH EAST


In [61]:
# Imprime a(s) linha(s) duplicadas
df_user = df_region[df_region.duplicated()].drop_duplicates()

In [62]:
df_drop = df_user[df_user['PRACTICE_NAME'].duplicated()].sort_values('PRACTICE_NAME',ascending=False)

In [63]:
df_drop.groupby(['REGIONAL_OFFICE_NAME','PRACTICE_NAME']).PRACTICE_NAME.size().sort_values(ascending=True)

REGIONAL_OFFICE_NAME      PRACTICE_NAME             
EAST OF ENGLAND           ABBEY ROAD SURGERY             1
NORTH WEST                HOLLIES MEDICAL CENTRE         1
                          HIGHFIELD SURGERY              1
                          HIGH STREET SURGERY            1
                          HIGH STREET MEDICAL CENTRE     1
                                                        ..
EAST OF ENGLAND           UNIDENTIFIED DOCTORS          12
SOUTH EAST                UNIDENTIFIED DOCTORS          13
MIDLANDS                  UNIDENTIFIED DOCTORS          15
NORTH EAST AND YORKSHIRE  UNIDENTIFIED DOCTORS          20
NORTH WEST                UNIDENTIFIED DOCTORS          23
Name: PRACTICE_NAME, Length: 389, dtype: int64

In [64]:
df_drop.groupby(['REGIONAL_OFFICE_NAME','PRACTICE_NAME']).PRACTICE_NAME.size().reset_index(name='PRESCRIBERS_BY_REGION').sort_values('REGIONAL_OFFICE_NAME',ascending=True)

,REGIONAL_OFFICE_NAME,PRACTICE_NAME,PRESCRIBERS_BY_REGION
0,EAST OF ENGLAND,ABBEY ROAD SURGERY,1
21,EAST OF ENGLAND,PARK MEDICAL CENTRE,1
22,EAST OF ENGLAND,PARKFIELD MEDICAL CENTRE,1
23,EAST OF ENGLAND,PARKSIDE MEDICAL CENTRE,1
24,EAST OF ENGLAND,PRIORY MEDICAL CENTRE,1
...,...,...,...
365,SOUTH WEST,CAMHS,1
364,SOUTH WEST,BIRCHWOOD MEDICAL PRACTICE,1
387,SOUTH WEST,WHITE HOUSE SURGERY,1
375,SOUTH WEST,PARK LANE PRACTICE,1


In [65]:
df_size = df_drop.groupby(['REGIONAL_OFFICE_NAME','PRACTICE_NAME']).PRACTICE_NAME.size().reset_index(name='PRESCRIBERS_BY_REGION').sort_values('PRACTICE_NAME',ascending=True)

In [66]:
# Ordenado por Prescriber 
df_size

,REGIONAL_OFFICE_NAME,PRACTICE_NAME,PRESCRIBERS_BY_REGION
99,MIDLANDS,ABBEY MEDICAL CENTRE,2
38,LONDON,ABBEY MEDICAL CENTRE,1
309,SOUTH EAST,ABBEY MEDICAL CENTRE,1
100,MIDLANDS,ABBEY MEDICAL PRACTICE,2
0,EAST OF ENGLAND,ABBEY ROAD SURGERY,1
...,...,...,...
174,MIDLANDS,WOODLANDS SURGERY,1
98,LONDON,WOODLANDS SURGERY,2
37,EAST OF ENGLAND,WOODLANDS SURGERY,1
308,NORTH WEST,WOODSIDE MEDICAL CENTRE,1


In [67]:
df_size['PRACTICE_NAME'].value_counts(ascending=True).reset_index(name='NR_PRESCRIBERS')

,index,NR_PRESCRIBERS
0,NEWBRIDGE SURGERY,1
1,PALLIATIVE CARE,1
2,PARK GREEN SURGERY,1
3,PARK LANE PRACTICE,1
4,PARK MEDICAL PRACTICE,1
...,...,...
269,COMMUNITY DERMATOLOGY SERVICE,5
270,RIVERSIDE SURGERY,5
271,HIGH STREET SURGERY,6
272,VILLAGE SURGERY,6


In [68]:
df_remove = df_size['PRACTICE_NAME'].value_counts(ascending=True).reset_index(name='NR_PRESCRIBERS')

In [69]:
df_repeated_precribers = df_remove[df_remove.NR_PRESCRIBERS != 1]

In [70]:
df_repeated_precribers

,index,NR_PRESCRIBERS
203,THE LIMES MEDICAL CENTRE,2
204,THE NEW SURGERY,2
205,CHURCH VIEW SURGERY,2
206,THE MEDICAL CENTRE,2
207,CHURCH STREET PRACTICE,2
...,...,...
269,COMMUNITY DERMATOLOGY SERVICE,5
270,RIVERSIDE SURGERY,5
271,HIGH STREET SURGERY,6
272,VILLAGE SURGERY,6


In [71]:
df_repeated_precribers_load = df_repeated_precribers.rename(columns={'index':'PRACTICE_NAME'})

In [72]:
df_repeated_precribers_load.merge(df_size,on='PRACTICE_NAME')

,PRACTICE_NAME,NR_PRESCRIBERS,REGIONAL_OFFICE_NAME,PRESCRIBERS_BY_REGION
0,THE LIMES MEDICAL CENTRE,2,MIDLANDS,3
1,THE LIMES MEDICAL CENTRE,2,SOUTH EAST,1
2,THE NEW SURGERY,2,LONDON,1
3,THE NEW SURGERY,2,SOUTH EAST,1
4,CHURCH VIEW SURGERY,2,EAST OF ENGLAND,1
...,...,...,...,...
181,UNIDENTIFIED DOCTORS,8,MIDLANDS,15
182,UNIDENTIFIED DOCTORS,8,LONDON,10
183,UNIDENTIFIED DOCTORS,8,NORTH EAST AND YORKSHIRE,20
184,UNIDENTIFIED DOCTORS,8,EAST OF ENGLAND,12


In [73]:
df_final = df_repeated_precribers_load.merge(df_size,on='PRACTICE_NAME')

In [74]:
df_final1 = df_final[['REGIONAL_OFFICE_NAME','PRACTICE_NAME']].groupby(['REGIONAL_OFFICE_NAME','PRACTICE_NAME'])

In [75]:
df_final1.head()

,REGIONAL_OFFICE_NAME,PRACTICE_NAME
0,MIDLANDS,THE LIMES MEDICAL CENTRE
1,SOUTH EAST,THE LIMES MEDICAL CENTRE
2,LONDON,THE NEW SURGERY
3,SOUTH EAST,THE NEW SURGERY
4,EAST OF ENGLAND,CHURCH VIEW SURGERY
...,...,...
181,MIDLANDS,UNIDENTIFIED DOCTORS
182,LONDON,UNIDENTIFIED DOCTORS
183,NORTH EAST AND YORKSHIRE,UNIDENTIFIED DOCTORS
184,EAST OF ENGLAND,UNIDENTIFIED DOCTORS


In [76]:
# ATENÇÃO: Informações solicitadas com o pedido no item f.
df_final1.head().sort_values('REGIONAL_OFFICE_NAME',ascending=True)

,REGIONAL_OFFICE_NAME,PRACTICE_NAME
38,EAST OF ENGLAND,MILL ROAD SURGERY
41,EAST OF ENGLAND,ORCHARD MEDICAL PRACTICE
129,EAST OF ENGLAND,COMMUNITY ENT SERVICE
46,EAST OF ENGLAND,OAKFIELD SURGERY
142,EAST OF ENGLAND,PARK MEDICAL CENTRE
...,...,...
56,SOUTH WEST,HILLCREST SURGERY
95,SOUTH WEST,KINGSWOOD SURGERY
31,SOUTH WEST,WHITE HOUSE SURGERY
76,SOUTH WEST,THE CEDARS SURGERY


In [77]:
df_regions_counts = df_final1.head().sort_values('REGIONAL_OFFICE_NAME',ascending=True)

In [78]:
# Número de todos os centros de saúde (Precribres) que estão em mais de uma região Classificação feita por região
df_regions_counts.value_counts('REGIONAL_OFFICE_NAME',ascending=False)

REGIONAL_OFFICE_NAME
MIDLANDS                    36
LONDON                      32
NORTH WEST                  31
SOUTH EAST                  26
EAST OF ENGLAND             25
NORTH EAST AND YORKSHIRE    25
SOUTH WEST                  10
UNIDENTIFIED                 1
dtype: int64

#### g.	Qual o preço médio dos químicos prescritos em no ultimo mês coletado?

In [79]:
df_202207.head(3)

,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,ICB_NAME,ICB_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,ADDRESS_1,...,BNF_CODE,BNF_DESCRIPTION,BNF_CHAPTER_PLUS_CODE,QUANTITY,ITEMS,TOTAL_QUANTITY,ADQUSAGE,NIC,ACTUAL_COST,UNIDENTIFIED
0,202206,NORTH WEST,Y62,NHS CHESHIRE AND MERSEYSIDE INTEGRATED C,QYG,WIRRAL COMMUNITY HEALTH AND CARE NHS FOU,RY700,WIRRAL COMMUNITY NMP,Y03836,ST CATHERINE'S HC,...,20020200701,Viscopaste PB7 bandage 7.5cm x 6m,20: Dressings,10.0,1,10.0,0.0,38.90,36.40326,N
1,202206,NORTH WEST,Y62,NHS CHESHIRE AND MERSEYSIDE INTEGRATED C,QYG,WIRRAL COMMUNITY HEALTH AND CARE NHS FOU,RY700,WIRRAL WIC (APH)_WIC APH,N85645,ARROWE PARK HOSPITAL,...,20030100079,Mepore dressing 11cm x 15cm,20: Dressings,5.0,1,5.0,0.0,1.85,1.74307,N
2,202206,NORTH EAST AND YORKSHIRE,Y63,NHS SOUTH YORKSHIRE INTEGRATED CARE BOAR,QF7,NHS NOTTINGHAM AND NOTTINGHAMSHIRE ICB -,02Q00,BASSETLAW HEALTH PARTNERSHIP,Y03762,C/O RETFORD HOSPITAL,...,20030100167,Dressit sterile dressing pack with gloves,20: Dressings,10.0,6,60.0,0.0,41.40,38.75440,N


In [80]:
df_202207['TOTAL_COST'] = df_202207['TOTAL_QUANTITY'] * df_202207['ACTUAL_COST']

In [81]:
df_202207.head(3)

,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,ICB_NAME,ICB_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,ADDRESS_1,...,BNF_DESCRIPTION,BNF_CHAPTER_PLUS_CODE,QUANTITY,ITEMS,TOTAL_QUANTITY,ADQUSAGE,NIC,ACTUAL_COST,UNIDENTIFIED,TOTAL_COST
0,202206,NORTH WEST,Y62,NHS CHESHIRE AND MERSEYSIDE INTEGRATED C,QYG,WIRRAL COMMUNITY HEALTH AND CARE NHS FOU,RY700,WIRRAL COMMUNITY NMP,Y03836,ST CATHERINE'S HC,...,Viscopaste PB7 bandage 7.5cm x 6m,20: Dressings,10.0,1,10.0,0.0,38.90,36.40326,N,364.03260
1,202206,NORTH WEST,Y62,NHS CHESHIRE AND MERSEYSIDE INTEGRATED C,QYG,WIRRAL COMMUNITY HEALTH AND CARE NHS FOU,RY700,WIRRAL WIC (APH)_WIC APH,N85645,ARROWE PARK HOSPITAL,...,Mepore dressing 11cm x 15cm,20: Dressings,5.0,1,5.0,0.0,1.85,1.74307,N,8.71535
2,202206,NORTH EAST AND YORKSHIRE,Y63,NHS SOUTH YORKSHIRE INTEGRATED CARE BOAR,QF7,NHS NOTTINGHAM AND NOTTINGHAMSHIRE ICB -,02Q00,BASSETLAW HEALTH PARTNERSHIP,Y03762,C/O RETFORD HOSPITAL,...,Dressit sterile dressing pack with gloves,20: Dressings,10.0,6,60.0,0.0,41.40,38.75440,N,2325.26400


In [82]:
"£{:,.2f}".format(df_202207['TOTAL_COST'].mean())

'£90,143.95'

#### h.	Gere uma tabela que contenha apenas a prescrição de maior valor de cada usuário.

In [83]:
df[['PRACTICE_NAME', 'ACTUAL_COST']].groupby('PRACTICE_NAME').max().head(8387)

,ACTUAL_COST
PRACTICE_NAME,
(FRACTURE CLINIC) NORTH OOH,1183.65046
(IRLAM) SALFORD CARE CTRS MEDICAL PRACTI,1294.30272
(OUT PATIENT DEPARTMENT) NORTH OOH,214.10748
0-19 EAST CHESHIRE HEALTH VISITORS,4.20343
0-19 PUBLIC HEALTH SERVICE HARTLEPOOL,5.46635
...,...
YOUR HEALTHCARE NON MED PRES,293.46451
YOXALL,2588.08465
ZAIN MEDICAL CENTRE,1243.10165
